# Calculator

## 1. 기능 구현

In [41]:
import os
import json
import numpy as np


# 함수.1
def extract_flattened_keypoints_from_json(file_path: str):
    """
    JSON 파일에서 Keypoints 값을 추출하고 평탄화하여 벡터로 변환하는 함수
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    keypoints = data.get("keypoints", [])
    flattened = [flattenend_data if flattenend_data is not None else 0 for kp in keypoints for flattenend_data in (kp['x'], kp['y'])]

    return flattened

# 함수.2
def convert_vectors_by_flattened(directory_path: str):
    """
    directory내 모든 JSON 파일에서 좌표 벡터를 수집하여 리스트로 반환하는 함수
    """
    # 저장소 만들기
    vectors = []
    files = os.listdir(directory_path)

    # 디렉토리 안에 JSON 데이터만 불러오기
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(directory_path, file)

            # 함수.1을 이용해 모든 파일의 좌표값을 flattened
            coords = extract_flattened_keypoints_from_json(file_path)

            if len(coords) == 34:
                vectors.append(np.array(coords))

    return vectors

# 함수.3
def cosine_similarity(vec1, vec2):
    """두 개의 벡터(vec1, vec2)의 코사인 유사도를 계산"""
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    
    # 벡터의 크기(Norm) 계산
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    # 두 벡터의 내적(dot product) 계산 후 코사인 유사도 반환
    similarity = np.dot(vec1, vec2) / (norm1 * norm2)
    
    return similarity

# 함수. 4
def compare_cosine_similarity(result1, result2):
    """
    result1과 result2의 코사인 유사도를 비교하는 함수
    result1과 result2는 각각 다차원 리스트(2D 배열) 형태일 수 있음.
    """
    similarities = []
    
    # 각 벡터를 1:1로 비교 (result1과 result2의 길이가 같다고 가정)
    for vec1, vec2 in zip(result1, result2):
        sim = cosine_similarity(vec1, vec2)
        similarities.append(sim)

    return similarities

# 함수. 5
def compare_cosine_similarity_matrix(answer_vectors, realtime_vectors):
    """
    정답 벡터 리스트와 실시간 벡터 리스트 간 모든 코사인 유사도를 비교

    Parameters:
    - answer_vectors: List[np.ndarray]  → 정답 데이터 벡터 리스트 (예: 87개)
    - realtime_vectors: List[np.ndarray]  → 실시간 데이터 벡터 리스트 (예: N개)

    Returns:
    - 2차원 리스트 (87 × N): 각 정답 벡터와 실시간 벡터 간 코사인 유사도 행렬
    """
    similarity_matrix = []

    for a_vec in answer_vectors:
        row = []
        for r_vec in realtime_vectors:
            # 벡터 유사도 계산 (예외 방지)
            norm1 = np.linalg.norm(a_vec)
            norm2 = np.linalg.norm(r_vec)
            if norm1 == 0 or norm2 == 0:
                sim = 0
            else:
                sim = np.dot(a_vec, r_vec) / (norm1 * norm2)
            row.append(sim)
        similarity_matrix.append(row)

    return similarity_matrix

Test

In [36]:
answer_directory_path1 = '../../LJH/data/front_json'
vec1 = convert_vectors_by_flattened(answer_directory_path1)
# print(vec1)

In [35]:
realtime_directory_path = '../../LJH/data/side_json'
vec2 = convert_vectors_by_flattened(realtime_directory_path)
# print(vec2)

In [40]:
print(vec1[0])
print(vec2[0])

[643 143 651 136 634 136 663 147 621 148 678 199 605 199 701 222 585 217
 668 216 621 213 670 331 618 332 685 438 604 440 697 544 592 546]
[646 148 653 140   0   0 674 148   0   0 667 195 685 197 608 218   0   0
 528 220   0   0 662 335 674 334 663 440 665 439 662 546 664 539]


## 2. 벡터 유사도 비교

In [43]:
# 코사인 유사도 분석 (각 이미지 비교: 87장)
cosines_result = compare_cosine_similarity(vec1, vec2)
print(cosines_result)

[0.896222333929139, 0.8963179051125744, 0.8959439891103309, 0.8963021991536727, 0.8964083120402385, 0.8961160656189991, 0.8959659720277089, 0.8957329166490843, 0.8953525446951409, 0.8954242207277585, 0.8952659282859463, 0.8950368193144558, 0.896356091556819, 0.8942874666345804, 0.8939169602835049, 0.894073844652035, 0.8936030375530971, 0.8932455239303001, 0.8930479038530725, 0.8927328549474051, 0.8930662598385946, 0.8926728797763944, 0.8925359341182454, 0.8953455235139842, 0.8893919585209921, 0.8917349657132988, 0.8919712464064745, 0.8918662638802289, 0.8920144992434821, 0.8919416903464545, 0.8917079491539268, 0.89175624353405, 0.891505352234042, 0.8914097187572833, 0.8922095246442758, 0.8850219150516534, 0.8910251246516983, 0.8911590217623161, 0.8910172257711257, 0.890882882286728, 0.8906264896052275, 0.8905371783618861, 0.890617125734384, 0.8903802143803531, 0.890501861879497, 0.8877644288658462, 0.8818326034061887, 0.8904227625735818, 0.8903754407146031, 0.8906124964497478, 0.890785

In [44]:
# 코사인 유사도 분석 (각 keypoints 하나의 값 모음)
cosines_result = compare_cosine_similarity_matrix(vec1, vec2)
print(cosines_result)

[[0.896222333929139, 0.8962040806789073, 0.8961990023083253, 0.8962352283040476, 0.8963036509384239, 0.8962568323452131, 0.8962762273240118, 0.8962373664381893, 0.8961979308683183, 0.8962586667719752, 0.8962705658146449, 0.8963479173693242, 0.8962197489015887, 0.896228767565213, 0.8961533733439775, 0.8960670544840255, 0.8960819677848528, 0.8960347308386359, 0.8958466261612695, 0.8958352378820239, 0.8957904049947457, 0.8957653983084843, 0.8956245326994909, 0.8952603062852411, 0.8961987470786115, 0.8948803919082489, 0.8945932467253577, 0.8945986890344737, 0.8942236141003246, 0.8938294562769636, 0.8934621988881727, 0.8928502489636787, 0.8928501576453539, 0.8925391557775555, 0.8920779008915191, 0.8962220550178029, 0.8914236554810976, 0.8909159036445752, 0.8907143187539944, 0.8900573265476793, 0.8894932852584575, 0.8890472294728666, 0.8886364970863011, 0.8885895660721861, 0.8881422879717098, 0.8877005231872337, 0.8962705955231715, 0.8872640462341439, 0.8869742516471969, 0.8869805748558215, 